
Circuit Simulation Example with OBI-One
======================================

This notebook demonstrates how to run circuit simulations using the obi-one simulation framework
with [BlueCelluLab](https://github.com/openbraininstitute/BlueCelluLab) as the simulator.

In [ ]:
import requests
import obi_one as obi
virtual_lab_id=obi.LAB_ID_STAGING_TEST
project_id=obi.PROJECT_ID_STAGING_TEST
from entitysdk import Client, ProjectContext

from obi_auth import get_token
from obi_notebook import get_projects

obi_one_api_url = "http://127.0.0.1:8100"

token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=obi.LAB_ID_STAGING_TEST, project_id=obi.PROJECT_ID_STAGING_TEST)
db_client = Client(api_url="https://staging.openbraininstitute.org/api/entitycore", project_context=project_context, token_manager=token)

Now, let's generate the Simulation Config

In [ ]:
from pathlib import Path
import obi_one as obi

# === Parameters ===
sim_duration = 3000.0

# === 1. Build Form (CircuitSimulationScanConfig) ===
sim_form = obi.CircuitSimulationScanConfig.empty_config()

# Info
info = obi.Info(
    campaign_name="Small Microcircuit Simulation",
    campaign_description="Simulation of circuit with predefined neuron set and constant current stimulus"
)
sim_form.set(info, name="info")

# Neuron Sets
sim_nset = obi.AllNeurons()
sim_form.add(sim_nset, name="All Biophys")

# Timestamps
timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=1, interval=100)
sim_form.add(timestamps, name="Timestamps")

# Stimulus
stimulus = obi.PoissonSpikeStimulus(
    duration=800.0,
    timestamps=timestamps.ref,
    frequency=20,
    source_neuron_set=sim_nset.ref,
    targeted_neuron_set=sim_nset.ref
)
sim_form.add(stimulus, name="PoissonInput")

stimulus = obi.ConstantCurrentClampSomaticStimulus(
    timestamps=timestamps.ref, duration=2000.0, neuron_set=sim_nset.ref, amplitude=0.5
)
sim_form.add(stimulus, name="CurrentClampInput")

# Recording
recording = obi.SomaVoltageRecording(neuron_set=sim_nset.ref)
sim_form.add(recording, name="SomaVoltage")

# Initialization block
init = obi.CircuitSimulationScanConfig.Initialize(
    circuit=obi.CircuitFromID(id_str="13a54362-0c99-43fd-94e4-58bdb5b79466"),
    node_set=sim_nset.ref,
    simulation_length=sim_duration,
)
sim_form.set(init, name="initialize")
# Validated Config
validated_sim_conf = sim_form.validated_config()

print(validated_sim_conf)

# === 2. Wrap into a Simulation ===
grid_scan = obi.GridScanGenerationTask(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='./obi-output/run_circuit_simulations/grid_scan')
grid_scan.execute(db_client=db_client)
obi.run_tasks_for_generated_scan(grid_scan, db_client=db_client)

In [ ]:
# population_name = circuit.default_population_name
simulation_config_path = grid_scan.single_configs[0].coordinate_output_root / "simulation_config.json"
print(simulation_config_path)

circuit_folder = grid_scan.single_configs[0].coordinate_output_root / "sonata_circuit"

In [ ]:
import json
from bluepysnap import Circuit, Simulation
from entitysdk import models
from entitysdk.client import Client
from entitysdk.staging.circuit import stage_circuit
from entitysdk.staging.simulation_result import stage_simulation, stage_simulation_result
from obi_auth import get_token
from obi_notebook import get_entities
from obi_notebook.get_environment import get_environment
from obi_notebook.get_projects import get_projects
from pathlib import Path
from tqdm import tqdm

# environment = get_environment.get_environment()
environment="staging"
token = get_token(environment=environment, auth_mode="daf")
project_context = get_projects(env=environment, token=token)
client = Client(environment=environment, project_context=project_context, token_manager=token)

In [ ]:
# Jim's test simulation
simulation_campaign_id = "84279f50-b454-4bb0-966b-2093e97f38db"
simulation_name = "Simulation 4"

# Pair: Inhibitory neruon spontaneously fires, driving positive EPSPs in excitatory neuron (with low membrane voltage)
simulation_campaign_id = "6adfcdda-d6d2-4151-a89c-bc1b7f474846"
simulation_name = "Simulation 0"

simulation_campaign = client.get_entity(entity_id=simulation_campaign_id, entity_type=models.SimulationCampaign)

In [ ]:
for sim in simulation_campaign.simulations:
    if sim.name == simulation_name:
        simulation = client.get_entity(entity_id=sim.id, entity_type=models.Simulation)
        simulation_config_path = stage_simulation(
                client=client,
                model=simulation,
                output_dir="./simulation",
            )
        break

circuit_folder = './simulation/circuit'

Get the path to the mod files and compile the mod files. 

In [ ]:
# Remove the old compiled mod files folder
! rm -r arm64/
# flag DISABLE_REPORTINGLIB to skip SonataReportHelper.mod and SonataReport.mod from compilation.
!../../.venv/bin/nrnivmodl -incflags "-DDISABLE_REPORTINGLIB" {circuit_folder}/mod

Run circuit simulation using BlueCelluLab backend. In future, we will support Neurodamus backend as well. This will run a SONATA simulation.  

In [ ]:
# Run the simulation
from obi_one.scientific.library.simulation_execution import run
run(
    simulation_config=simulation_config_path,
    simulator="bluecellulab", # Optional: bluecellulab or neurodamus. Default: bluecellulab
    save_nwb=True            # Optional: Save results in NWB format. Default: False
)

The results are stored in the `output` directory. The logs are also stored in the `logs` in notebooks folder.

### Spike Report Analysis using BluePySnap

We will use the example [notebook](https://github.com/openbraininstitute/snap/blob/master/doc/source/notebooks/06_spike_reports.ipynb) from BluePySnap to analyse the spike report generated by the circuit simulation.

In [ ]:
import bluepysnap

snap_simulation = bluepysnap.Simulation(simulation_config_path)
spikes = snap_simulation.spikes
print(
    spikes.time_start,
    spikes.time_stop,
    spikes.dt
)
print(spikes.population_names)

In [ ]:
spike_pop = spikes[spikes.population_names[0]]
node_population = spike_pop.nodes

In [ ]:
filtered = spikes.filter(group={}, t_start=spikes.time_start, t_stop=spikes.time_stop)
# filtered = spikes.filter(group={'layer':'5'}, t_start=spikes.time_start, t_stop=spikes.time_stop)
filtered.report.head()

In [ ]:
filtered.raster();

In [ ]:
filtered.firing_rate_histogram();

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

ax = plt.gca()
ax.set_xlabel("Time [ms]")
ax.set_ylabel("PSTH [Hz]")
ax.set_title(f"PSTH for group: {filtered.group}")

times = filtered.report.index

time_start = np.min(times)
time_stop = np.max(times)

# heuristic for a nice bin size (~100 spikes per bin on average)
time_binsize = min(50.0, (time_stop - time_start) / ((len(times) / 100.0) + 1.0))

bins = np.append(np.arange(time_start, time_stop, time_binsize), time_stop)
hist, bin_edges = np.histogram(times, bins=bins)
node_count = len(snap_simulation.circuit.nodes.ids(filtered.group))  # Get length of node ids for whole `group`
freq = 1.0 * hist / node_count / (0.001 * time_binsize)

# use the middle of the bins instead of the start of the bin
ax.plot(0.5 * (bin_edges[1:] + bin_edges[:-1]), freq, label="PSTH", drawstyle="steps-mid");

In [ ]:
spikes.filter().raster();

In [ ]:
spikes.filter().raster(y_axis='etype');

In [ ]:
spikes.filter().isi(binsize=20);

### Soma Report Analysis

Let's [load](https://github.com/openbraininstitute/snap/blob/master/doc/source/notebooks/07_frame_reports.ipynb) the soma report using BluePySnap and plot it.

In [ ]:
snap_simulation.reports

In [ ]:
soma_report = snap_simulation.reports['SomaVoltage']

In [ ]:
print(
    soma_report.time_start,
    soma_report.time_stop,
    soma_report.dt
)  # Gives a warning in case the dt differs from simulation.dt

In [ ]:
soma_report.population_names

In [ ]:
soma_pop = soma_report[soma_report.population_names[0]]

In [ ]:
node_population = soma_pop.nodes

In [ ]:
ids = soma_pop.node_ids
node_population.get(ids, properties=['layer','synapse_class','x','y','z']).head()

In [ ]:
filtered = soma_report.filter(group={'layer':'5'}, t_start= soma_report.time_start, t_stop= soma_report.time_stop)
filtered.report.head()